In [1]:
import pandas as pd
import requests
import json
import ast
from bs4 import BeautifulSoup

In [2]:
restaurant_data_df = pd.read_excel("rw_resturants_2025-07_v2.xlsx")
restaurant_data_df

,borough,neighborhood,primaryCategory,primaryLocation,ctaButtons,tags,shortTitle,slug,image,summary,website,menuFileUrl,collections,mealTypes,restaurantInclusionWeek,ecommerce,rest_week_site_url,image_url
0,Manhattan,Midtown East,Restaurants,Midtown East,"{'primary': {'label': {'presale': False, 'gene...","['Italian', 'Midtown East']",Casa Lever,casa-lever,{'url': '//images.ctfassets.net/1aemqu6a6t65/5...,"At Casa Lever, Modern art and timeless archite...",https://www.casalever.com,NaN,[],['$60 Dinner Price'],"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",NaN,https://www.nyctourism.com/restaurant-week/cas...,https://images.ctfassets.net/1aemqu6a6t65/5bH2...
1,Manhattan,West Village,Restaurants,West Village,"{'primary': {'label': {'presale': False, 'gene...","['Austrian', 'West Village']",Wallsé,wallse,{'url': '//images.ctfassets.net/1aemqu6a6t65/4...,This modern European restaurant offers an impe...,https://www.wallse.com,NaN,[],['$60 Dinner Price'],"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",NaN,https://www.nyctourism.com/restaurant-week/wallse,https://images.ctfassets.net/1aemqu6a6t65/4dNX...
2,Brooklyn,Park Slope,Restaurants,Park Slope,"{'primary': {'label': {'presale': False, 'gene...","['Japanese / Sushi', 'Park Slope']",Muteki Udon,muteki-udon,{'url': '//images.ctfassets.net/1aemqu6a6t65/C...,Visit this Park Slope restaurant for an unforg...,https://mutekiudon.com,NaN,[],"['$45 Lunch Price', '$60 Dinner Price', '$45 S...","['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",NaN,https://www.nyctourism.com/restaurant-week/mut...,https://images.ctfassets.net/1aemqu6a6t65/CCK7...
3,Manhattan,Flatiron District,Restaurants,Flatiron District,"{'primary': {'label': {'presale': False, 'gene...","['Japanese / Sushi', 'Flatiron District']",Aqua New York,aqua-new-york,{'url': '//images.ctfassets.net/1aemqu6a6t65/4...,Aqua New York has two separate restaurants—Aqu...,https://aqua-nyc.com,NaN,[],"['$30 Lunch Price', '$60 Dinner Price', '$60 S...","['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",NaN,https://www.nyctourism.com/restaurant-week/aqu...,https://images.ctfassets.net/1aemqu6a6t65/4uSd...
4,Manhattan,Lower East Side,Restaurants,Lower East Side,"{'primary': {'label': {'presale': False, 'gene...","['American (New)', 'Lower East Side']",Essex,essex,{'url': '//images.ctfassets.net/1aemqu6a6t65/4...,Located in the historic Essex Street Market Bu...,https://www.essexnyc.com,NaN,[],"['$45 Dinner Price', '$30 Sunday Lunch/Brunch ...","['Week 1 (July 21 - July 27)', 'Week 2 (July 2...","{'title': 'OpenTable', 'partnerName': 'OpenTab...",https://www.nyctourism.com/restaurant-week/essex,https://images.ctfassets.net/1aemqu6a6t65/4oIY...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,Staten Island,Todt Hill,Restaurants,Todt Hill,"{'primary': {'label': {'presale': False, 'gene...","['Continental', 'Todt Hill']",Max's Esca,maxs-esca,"{'url': '', 'alt': ''}",Max's Esca is a premier destination for except...,https://www.maxsesca.com,NaN,[],['$60 Dinner Price'],"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",NaN,https://www.nyctourism.com/restaurant-week/max...,NaN
607,Brooklyn,Williamsburg,Restaurants,Williamsburg,"{'primary': {'label': {'presale': False, 'gene...","['American (New)', 'Williamsburg']",Sungold,sungold,"{'url': '', 'alt': ''}","With an eye on the Mediterranean, chef Michael...",https://www.sungoldbk.com,NaN,[],"['$30 Lunch Price', '$45 Dinner Price', '$30 S...","['Week 1 (July 21 - July 27)', 'Week 2 (July 2...","{'title': 'OpenTable', 'partnerName': 'OpenTab...",https://www.nyctourism.com/restaurant-week/sun...,NaN
608,Brooklyn,Williamsburg,Nightlife,Williamsburg,"{'primary': {'label': {'presale': False, 'gene...","['Italian', 'Williamsburg']",Little Fino,little-fino,"{'url': '', 'alt': ''}",Little Fino is an all day neighborhood bar and...,https://www.littlefinonyc.com,NaN,[],['$30 Lunch Price'],"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...","{'title': 'OpenTable', 'part

In [3]:
def get_page_data(rest_week_site_url):
    page_data = {}
    req = requests.get(rest_week_site_url)
    if req.status_code != 200:
        return {"location_coords": {"lat": "", "lon": ""}, "pdf_url": "", "venueAddress": ""}
        
    soup = BeautifulSoup(req.text)
    next_data = soup.select("#__NEXT_DATA__")[0]
    next_data_json = json.loads(next_data.text)
    fields = next_data_json["props"]["pageProps"]["pageFields"]["fields"]
    page_data["location_coords"] = fields["location"]
    page_data["pdf_url"] = next_data_json["props"]["pageProps"]["page"].get("fileUrl", "")
    page_data["venueAddress"] = fields["venueAddress"]

    return page_data

In [4]:
def get_all_rest_data(search_data):
    page_data = get_page_data(search_data["rest_week_site_url"])
    
    rest_data = {}
    rest_data["name"] = search_data["shortTitle"]
    rest_data["pdf_url"] = page_data["pdf_url"]
    rest_data["slug"] = search_data["slug"]
    rest_data["url"] = search_data["rest_week_site_url"]
    rest_data["venueAddress"] = page_data["venueAddress"]
    rest_data["latitude"] = page_data["location_coords"]["lat"]
    rest_data["longitude"] = page_data["location_coords"]["lon"]
    rest_data["summary"] = search_data["summary"]
    rest_data["website"] = search_data["website"]
    rest_data["collections"] = search_data["collections"]
    rest_data["borough"] = search_data["borough"]
    rest_data["neighborhood"] = search_data["neighborhood"]
    rest_data["primaryCategory"] = search_data["primaryCategory"]
    rest_data["primaryLocation"] = search_data["primaryLocation"]
    rest_data["restaurantInclusionWeek"] = search_data["restaurantInclusionWeek"]
    rest_data["image_url"] = search_data["image_url"]
    
    mealTypes = ast.literal_eval(search_data["mealTypes"])
    rest_data["meal_type"] = "; ".join(mealTypes).replace(" Price", "")

    rest_tags = ast.literal_eval(search_data["tags"].replace("None", '""'))
    rest_data["tags"] = "; ".join(rest_tags).strip("; ")

    return rest_data

In [5]:
%%time
count = 1
total = len(restaurant_data_df)
all_rest_data = []
for index, restaurant in restaurant_data_df.iterrows():
    print(f"{count} out of {total}: {restaurant['shortTitle']}")
    rest_data = get_all_rest_data(restaurant)
    all_rest_data.append(rest_data)
    count += 1

1 out of 611: Casa Lever
2 out of 611: Wallsé
3 out of 611: Muteki Udon
4 out of 611: Aqua New York
5 out of 611: Essex
6 out of 611: De Mole - Williamsburg
7 out of 611: The Orchard Townhouse
8 out of 611: Fish Cheeks
9 out of 611: Le Pavillon
10 out of 611: Benny John's Bar and Grill
11 out of 611: Hunt & Fish Club NYC
12 out of 611: Alice
13 out of 611: No Work Allowed
14 out of 611: Morandi
15 out of 611: Riverpark
16 out of 611: Mamo
17 out of 611: Maiella
18 out of 611: RA Sushi Bar Restaurant – Times Square
19 out of 611: Metropolis
20 out of 611: Psaraki
21 out of 611: DiWine
22 out of 611: Robert
23 out of 611: AperiBar
24 out of 611: Bixi
25 out of 611: Café Boulud
26 out of 611: Gyu-Kaku Japanese BBQ - Amsterdam
27 out of 611: Mishik
28 out of 611: Sarabeth's Central Park South
29 out of 611: Dagon
30 out of 611: A La Turka
31 out of 611: Torch & Crown Brewing Company
32 out of 611: Gaonnuri
33 out of 611: Draught 55
34 out of 611: Manuela
35 out of 611: Eléa
36 out of 611: 

264 out of 611: La Gioconda
265 out of 611: Bar Milano by Eataly
266 out of 611: Pardon My French
267 out of 611: Kaizen
268 out of 611: Giorgio's of Gramercy
269 out of 611: Fresco by Scotto
270 out of 611: Estiatorio Milos Hudson Yards
271 out of 611: Limani
272 out of 611: Manhatta
273 out of 611: Lido
274 out of 611: Indochine
275 out of 611: State Grill and Bar
276 out of 611: Dagg Thai
277 out of 611: Animo!
278 out of 611: The Bronx Beer Hall
279 out of 611: Jazba
280 out of 611: Wonderland Bar
281 out of 611: Il Buco Alimentari & Vineria
282 out of 611: Junoon
283 out of 611: Jacques Brasserie
284 out of 611: Mission Ceviche - Union Square
285 out of 611: Haven Rooftop
286 out of 611: Maizal
287 out of 611: Dowling’s at The Carlyle
288 out of 611: Rynn Thai Restaurant & Bar
289 out of 611: Meet the Meat
290 out of 611: Charlie Palmer Steak IV
291 out of 611: Boucherie West Village
292 out of 611: Oceans
293 out of 611: La Pecora Bianca Midtown
294 out of 611: Dear Strangers
295

513 out of 611: Benihana
514 out of 611: Dos Caminos Lexington
515 out of 611: Akoya
516 out of 611: Vestry
517 out of 611: Bourbon Steak New York
518 out of 611: Dhamaka
519 out of 611: Raf's
520 out of 611: Pinky Swear
521 out of 611: Noodle Bar - Uptown
522 out of 611: See No Evil Pizza
523 out of 611: Amélie - West Village
524 out of 611: Wildflower
525 out of 611: Le Monde
526 out of 611: Cha Cha Tang
527 out of 611: Paris Bar
528 out of 611: Cathédrale
529 out of 611: Laut
530 out of 611: The Dynamo Room
531 out of 611: Wayward Fare
532 out of 611: Altair
533 out of 611: Mamazul
534 out of 611: Patrick's on the Hill
535 out of 611: Anassa Taverna - Downtown
536 out of 611: Barlume
537 out of 611: Yefsi Estiatorio
538 out of 611: Sarabeth's Greenwich Village
539 out of 611: Masalawala & Sons
540 out of 611: Atlantis Prime
541 out of 611: Leonetta
542 out of 611: Barawine
543 out of 611: Tonchin Brooklyn
544 out of 611: Gyu-Kaku Japanese BBQ – Flushing
545 out of 611: Felice Columb

In [6]:
all_rest_data_df = pd.DataFrame(all_rest_data)
all_rest_data_df

,name,pdf_url,slug,url,venueAddress,latitude,longitude,summary,website,collections,borough,neighborhood,primaryCategory,primaryLocation,restaurantInclusionWeek,image_url,meal_type,tags
0,Casa Lever,https://nyc-tourism-public.s3.amazonaws.com/sr...,casa-lever,https://www.nyctourism.com/restaurant-week/cas...,"390 Park Ave.,Manhattan,10022,NY",40.759733,-73.972952,"At Casa Lever, Modern art and timeless archite...",https://www.casalever.com,[],Manhattan,Midtown East,Restaurants,Midtown East,"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",https://images.ctfassets.net/1aemqu6a6t65/5bH2...,$60 Dinner,Italian; Midtown East
1,Wallsé,https://nyc-tourism-public.s3.amazonaws.com/sr...,wallse,https://www.nyctourism.com/restaurant-week/wallse,"344 W. 11th St.,Manhattan,10014,NY",40.735425,-74.008236,This modern European restaurant offers an impe...,https://www.wallse.com,[],Manhattan,West Village,Restaurants,West Village,"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",https://images.ctfassets.net/1aemqu6a6t65/4dNX...,$60 Dinner,Austrian; West Village
2,Muteki Udon,https://nyc-tourism-public.s3.amazonaws.com/sr...,muteki-udon,https://www.nyctourism.com/restaurant-week/mut...,"234 Flatbush Ave.,Brooklyn,11217,NY",40.680372,-73.97514,Visit this Park Slope restaurant for an unforg...,https://mutekiudon.com,[],Brooklyn,Park Slope,Restaurants,Park Slope,"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",https://images.ctfassets.net/1aemqu6a6t65/CCK7...,$45 Lunch; $60 Dinner; $45 Sunday Lunch/Brunch...,Japanese / Sushi; Park Slope
3,Aqua New York,https://nyc-tourism-public.s3.amazonaws.com/sr...,aqua-new-york,https://www.nyctourism.com/restaurant-week/aqu...,"902 Broadway,Manhattan,10010,NY",40.739372,-73.989405,Aqua New York has two separate restaurants—Aqu...,https://aqua-nyc.com,[],Manhattan,Flatiron District,Restaurants,Flatiron District,"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",https://images.ctfassets.net/1aemqu6a6t65/4uSd...,$30 Lunch; $60 Dinner; $60 Sunday Dinner,Japanese / Sushi; Flatiron District
4,Essex,https://nyc-tourism-public.s3.amazonaws.com/sr...,essex,https://www.nyctourism.com/restaurant-week/essex,"124 Rivington St.,Manhattan,10002,NY",40.719918,-73.987059,Located in the historic Essex Street Market Bu...,https://www.essexnyc.com,[],Manhattan,Lower East Side,Restaurants,Lower East Side,"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",https://images.ctfassets.net/1aemqu6a6t65/4oIY...,$45 Dinner; $30 Sunday Lunch/Brunch; $45 Sunda...,American (New); Lower East Side
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,Max's Esca,,maxs-esca,https://www.nyctourism.com/restaurant-week/max...,"1559 Richmond Rd.,Staten Island,10304,NY",40.591501,-74.101216,Max's Esca is a premier destination for except...,https://www.maxsesca.com,[],Staten Island,Todt Hill,Restaurants,Todt Hill,"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",NaN,$60 Dinner,Continental; Todt Hill
607,Sungold,,sungold,https://www.nyctourism.com/restaurant-week/sun...,"Arlo Williamsburg, 96 Wythe Ave.,Brooklyn,1124...",40.721505,-73.958702,"With an eye on the Mediterranean, chef Michael...",https://www.sungoldbk.com,[],Brooklyn,Williamsburg,Restaurants,Williamsburg,"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",NaN,$30 Lunch; $45 Dinner; $30 Sunday Lunch/Brunch...,American (New); Williamsburg
608,Little Fino,https://nyc-tourism-public.s3.amazonaws.com/sr...,little-fino,https://www.nyctourism.com/restaurant-week/lit...,"111 N. 12th St.,Brooklyn,11249,NY",39.954235,-75.159411,Little Fino is an all day neighborhood bar and...,https://www.littlefinonyc.com,[],Brooklyn,Williamsburg,Nightlife,Williamsburg,"['Week 1 (July 21 - July 27)', 'Week 2 (July 2...",NaN,$30 Lunch,Italian; Williamsburg
609,Asuka Sushi,,asuka-sushi,https://www.nyctourism.com/restaurant-week/asu...,"300 W. 23rd St.,Manhattan,10011,NY",40.745218,-73.998929,"From various rolls to teriyaki, and many mouth...",https://www.asukasushiny.com,[],Manhattan

In [7]:
all_rest_data_df.to_excel("rw_resturants_2025-07_full.xlsx", index=False)